[BPE Blog](https://sebastianraschka.com/blog/2025/bpe-from-scratch.html)

[Research Paper](https://aclanthology.org/P16-1162.pdf)

In [ ]:
from collections import Counter
import json

In [ ]:
class BytePairEncodingTokenizer:
  def __init__(self,word_split = 'Ġ'):
    self.vocab={}
    self.inverse_vocab={}
    self.tokens=[]
    self.bpe_pairs={}
    self.word_split = word_split

  def set_vocabulary(self,word_split,special_tokens):
    # Step 1: Setting the unique characters
    unique_characters=[chr(i) for i in range(256)]

    # Step 2: Word split characters
    if word_split not in unique_characters:
      unique_characters.append(self.word_split)

    # Step 3: Special tokens
    if special_tokens:
      unique_characters.extend(self.special_tokens)

    return unique_characters

  def get_max_freq_pair(self,tokens):
    pair_counts={}
    pairs=[]
    # Step 1: Gettkng all the token pairs-> (token[i],token[i+1])
    for index in range(len(tokens)-1):
      pairs.append((tokens[index],tokens[index+1]))

    # Step 2: Getting the count of occurences of each of token pairs
    pairs_counts=Counter(pairs)

    # Step 3: Get the token pair whose count is the highest
    max_pair=max(pairs_counts.items(),key=lambda x: x[1])[0]
    return max_pair

  def merge_tokens(self,tokens,max_pair,new_pair_id):
    # In the tokens, check the presence of occurence of max_pair and if exists then replace max_pair with new_pair_id
    # Eg: tokens=[87,76,44,25,38,44,25,19], max_pair=[44,25],  new_pair_id=123
    # Output: [87,76,123,38,123,19]
    new_tokens=[]
    i=0
    while i<=len(tokens)-1:
      if i==len(tokens)-1:
        new_tokens.append(tokens[i])
        break

      elif (tokens[i],tokens[i+1])==max_pair:
        new_tokens.append(new_pair_id)
        i+=2

      else:
        new_tokens.append(tokens[i])
        i+=1
    return new_tokens

  def train(self,text,vocab_size,special_tokens):
    if vocab_size<=258:
      raise ValueError('Please enter a vocab size greater than 258 since this defines the basic set of characters')
    self.special_tokens = special_tokens

    # Setting the vocabulary
    vocab = self.set_vocabulary(self.word_split,self.special_tokens)
    for index,character in enumerate(vocab):
      self.vocab[index]=character
      self.inverse_vocab[character]=index

    # Transforming the text
    ## Step 1: Replacing all thw white-space character
    processed_text=[]
    for index,char in enumerate(text):
      if index!=0 and char==' ':
        processed_text.append(self.word_split)
      if char!=' ':
        processed_text.append(char)
    processed_text="".join(processed_text)

    ## Step 2: Getting the numerical form of token
    self.tokens=[]
    for char in processed_text:
      self.tokens.append(self.inverse_vocab[char])

    ## Step 3: BPE-algorithm
    vocab_length=len(self.vocab)
    for i in range(vocab_length,vocab_size):
      max_pair=self.get_max_freq_pair(self.tokens)
      if max_pair is None:
        break
      self.bpe_pairs[max_pair]=i
      self.tokens=self.merge_tokens(self.tokens,max_pair,i)

    ## Step 4: Update vocab with BPE
    for pair,new_index in self.bpe_pairs.items():
      merged_token=self.vocab[pair[0]]+self.vocab[pair[1]]
      self.vocab[new_index]=merged_token
      self.inverse_vocab[merged_token]=new_index

  def encode(self,text):
    # Step 1: Basically tokens are split into words. Replace all the occurences of "\n" to " <NEWLINE> ". This is to avoid splitting issues.
    tokens_split=text.replace('\n',' <NEWLINE> ').split()
    tokens=[]
    for i in tokens_split:
      if i=='<NEWLINE>':
        tokens.append('\n')
      else:
        tokens.append(i)

    # Step 2: Cleaning of tokens
    ## Eg: 'This is a ball' will be tokenized as ['The','Ġis','Ġa', 'Ġball']
    # Ensures that all the tokens in a line other than the first one will be prefixed with "Ġ" to show the word boundaries
    tokens_cleaned=[]
    for index,token in enumerate(tokens):
      if index>0 and not token.startswith('\n'):
        tokens_cleaned.append(self.word_split+token)
      else:
        tokens_cleaned.append(token)

    # Step 3: Getting the corresponding token IDs from the cleaned tokens
    ## Checks whether tokens exist in the vocabulary. If not, then perform BPE tokenization of the token
    token_ids=[]
    for token in tokens_cleaned:
      if token in self.inverse_vocab.keys():
        token_ids.append(self.inverse_vocab[token])
      else:
        token_ids.extend(self.tokenize_using_bpe(token))
    return token_ids

  def tokenize_using_bpe(self,token):
    # Step 1: Mapping the tokens to their IDs from the vocabulary
    token_ids=[]
    for char in token:
      if char in self.inverse_vocab.keys():
        token_ids.append(self.inverse_vocab[char])
      else:
        token_ids.append(None)

    # Step 2: Check whether token does not exist in Vocabulary- In that case stop
    if None in token_ids:
      token_dict=dict(zip(token_ids,token))
      missing_characters=[]
      for id,ch in token_dict.items():
        if id is None:
          missing_characters.append(ch)
      raise ValueError(f"No token IDs found for the characters:{missing_characters}")

    # Step 3: Now merging
    can_merge=True
    while can_merge and len(token_ids)>1:
      can_merge=False
      i=0
      new_tokens=[]
      """
      Check whether the token pair is part of bpe_pairs occured during training,
      If yes, index = index + 2, else index = index + 1.
      This iteration occurs until there exists no merging exists for all the tokens in token_ids.
      No merging exists means that there are no more possible keys to merge in bpe_pairs.
      """
      while i<len(token_ids)-1:
        pair=(token_ids[i],token_ids[i+1])
        if pair in self.bpe_pairs.keys():
          pair_id=self.bpe_pairs[pair]
          new_tokens.append(pair_id)
          i+=2
          can_merge=True
        else:
          new_tokens.append(token_ids[i])
          i+=1
      if i<len(token_ids):
        new_tokens.append(token_ids[i])
      token_ids=new_tokens

    return token_ids

  def decode(self,token_ids):
    # Step 1: Check whether there are non-existing token IDs
    non_existing_ids=[]
    for id in token_ids:
      if id not in self.vocab.keys():
        non_existing_ids.append(id)
    if len(non_existing_ids)>0:
      raise ValueError(f"No token found for the token IDs:{non_existing_ids}")

    # Step 2: Decoding- Check whether text corresponding to token ID starts with word_split-symbol('Ġ'). If yes replace word_split-symbol with " " else just append the text to string
    final=""
    for id in token_ids:
      text=self.vocab[id]
      if text.startswith(self.word_split):
        final+=" "+text[1:]
      else:
        final+=""+text

    return final

  def save_bpe_vocab_and_merges(self,vocab_path,bpe_path):
    with open(vocab_path,'w',encoding='utf-8') as f:
      json.dump(self.vocab,f,ensure_ascii=False, indent=2)
    with open(bpe_path,'w',encoding='utf-8') as f:
      json.dump([{'pair':list(pair),'id':id } for pair,id in self.bpe_pairs.items()],f,
                ensure_ascii=False, indent=2)

  def load_bpe_vocab_and_merges(self,vocab_path,bpe_path):
    with open(vocab_path,'r',encoding='utf-8') as f:
      loaded_vocab=json.load(f)
      self.vocab = {int(id):token for id,token in loaded_vocab.items()}
      self.inverse_vocab={token:int(id) for id,token in self.vocab.items()}
    with open(bpe_path,'r',encoding='utf-8') as f:
      bpe=json.load(f)
      for merge in bpe:
        self.bpe_pairs[tuple(merge['pair'])]=merge['id']


In [ ]:
import os
import urllib.request

if not os.path.exists("the-verdict.txt"):
    url = ("https://raw.githubusercontent.com/rasbt/"
           "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
           "the-verdict.txt")
    file_path = "the-verdict.txt"
    urllib.request.urlretrieve(url, file_path)

with open("the-verdict.txt", "r", encoding="utf-8") as f:
    text = f.read()

In [ ]:
tokenizer1 = BytePairEncodingTokenizer()
tokenizer1.train(text, vocab_size=259, special_tokens={"<|endoftext|>","œ"})

In [ ]:
tokenizer1.vocab

{0: '\x00',
 1: '\x01',
 2: '\x02',
 3: '\x03',
 4: '\x04',
 5: '\x05',
 6: '\x06',
 7: '\x07',
 8: '\x08',
 9: '\t',
 10: '\n',
 11: '\x0b',
 12: '\x0c',
 13: '\r',
 14: '\x0e',
 15: '\x0f',
 16: '\x10',
 17: '\x11',
 18: '\x12',
 19: '\x13',
 20: '\x14',
 21: '\x15',
 22: '\x16',
 23: '\x17',
 24: '\x18',
 25: '\x19',
 26: '\x1a',
 27: '\x1b',
 28: '\x1c',
 29: '\x1d',
 30: '\x1e',
 31: '\x1f',
 32: ' ',
 33: '!',
 34: '"',
 35: '#',
 36: '$',
 37: '%',
 38: '&',
 39: "'",
 40: '(',
 41: ')',
 42: '*',
 43: '+',
 44: ',',
 45: '-',
 46: '.',
 47: '/',
 48: '0',
 49: '1',
 50: '2',
 51: '3',
 52: '4',
 53: '5',
 54: '6',
 55: '7',
 56: '8',
 57: '9',
 58: ':',
 59: ';',
 60: '<',
 61: '=',
 62: '>',
 63: '?',
 64: '@',
 65: 'A',
 66: 'B',
 67: 'C',
 68: 'D',
 69: 'E',
 70: 'F',
 71: 'G',
 72: 'H',
 73: 'I',
 74: 'J',
 75: 'K',
 76: 'L',
 77: 'M',
 78: 'N',
 79: 'O',
 80: 'P',
 81: 'Q',
 82: 'R',
 83: 'S',
 84: 'T',
 85: 'U',
 86: 'V',
 87: 'W',
 88: 'X',
 89: 'Y',
 90: 'Z',
 91: '[',


In [ ]:
print(len(tokenizer1.vocab))

450


In [ ]:
print(len(tokenizer1.bpe_pairs))

742


In [ ]:
input_text = "Jack embraced beauty through art and life."
token_ids = tokenizer1.encode(input_text)
print(token_ids)

[425, 256, 101, 109, 98, 334, 99, 312, 256, 297, 97, 117, 116, 121, 304, 336, 117, 307, 288, 114, 116, 288, 110, 100, 256, 327, 102, 101, 46]


In [ ]:
print(tokenizer1.decode(token_ids))

Jack embraced beauty through art and life.


In [ ]:
for id in token_ids:
  print(id,'->',tokenizer1.decode([id]))

425 -> Jack
256 ->  
101 -> e
109 -> m
98 -> b
334 -> ra
99 -> c
312 -> ed
256 ->  
297 -> be
97 -> a
117 -> u
116 -> t
121 -> y
304 ->  th
336 -> ro
117 -> u
307 -> gh
288 ->  a
114 -> r
116 -> t
288 ->  a
110 -> n
100 -> d
256 ->  
327 -> li
102 -> f
101 -> e
46 -> .


In [ ]:
tokenizer1.encode("This is some text.")

[84, 278, 115, 256, 300, 322, 306, 101, 260, 101, 120, 116, 46]

In [ ]:
tokenizer1.encode('ജis')

ValueError: No token IDs found for the characters:['ജ']

In [ ]:
tokenizer1.decode([542, 299, 256, 299, 321, 305, 101, 259, 461, 116, 5000])

ValueError: No token found for the token IDs:[5000]

In [ ]:
tokenizer1.decode(tokenizer1.encode("This is some text."))

'This is some text.'

In [ ]:
tokenizer1.save_bpe_vocab_and_merges('vocab.json','bpe.json')

In [ ]:
tokenizer2=BytePairEncodingTokenizer()
tokenizer2.load_bpe_vocab_and_merges('vocab.json','bpe.json')

In [ ]:
tokenizer2.encode("This is some text.")

[542, 299, 256, 299, 321, 305, 101, 259, 461, 116, 46]

In [ ]:
tokenizer2.decode([542, 299, 256, 299, 321, 305, 101, 259, 461, 116, 46])

'This is some text.'

In [ ]:
len(tokenizer2.vocab)

1000

In [ ]:
len(tokenizer2.bpe_pairs)

742